In [2]:
import MySQLdb
import sshtunnel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from scipy.stats import pearsonr
from ast import literal_eval
import scipy.stats as stats

In [3]:
sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0

with sshtunnel.SSHTunnelForwarder(
    ('ssh.pythonanywhere.com'),
    ssh_username='Grawi', ssh_password='midgeq-xikFah-gukpu7',
    remote_bind_address=('Grawi.mysql.pythonanywhere-services.com', 3306)
) as tunnel:
    print("Successfully connected to Pythonanywhere")
    connection = MySQLdb.connect(
        user='Grawi',
        passwd='david2202087',
        host='127.0.0.1', port=tunnel.local_bind_port,
        db='Grawi$Interactive_quiz_database',
    )
    # Do stuff
    print("Successfully connected to database")
    
    try:
        cursor = connection.cursor()
        df = [None, None, None]
        df[0] = pd.read_sql("SELECT * FROM condition_1;", connection)
        df[1] = pd.read_sql("SELECT * FROM condition_2;", connection)
        df[2] = pd.read_sql("SELECT * FROM condition_3;", connection)
        print(f"Data read finished, length: condition 1 length {len(df[0])}, condition 2 length {len(df[1])}, condition 3 length {len(df[2])}")
    except Exception as e:
        print("Error:", e)
    
    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()

Successfully connected to Pythonanywhere
Successfully connected to database
Data read finished, length: condition 1 length 99, condition 2 length 10, condition 3 length 10


### Attention Pass Check

In [42]:
for i in range(1):
    df[i]['additional_answers'] = df[i]['additional_answers'].apply(literal_eval)
    df[i]['non_ideology_answers'] = df[i]['non_ideology_answers'].apply(literal_eval)
    if hasattr(df[i], 'ideology_answers'):
        df[i]['ideology_answers'] = df[i]['ideology_answers'].apply(literal_eval)


ValueError: malformed node or string: [[1, -1.7, -1]]

In [ ]:


for i in range(3):
    id_attention_failed = df[i][df[i]['attention_passed'] == 0]['participantId'].tolist()
    print(f"For condition {i + 1}, {len(id_attention_failed)} participants failed attention passed: {id_attention_failed}")
    df[i] = df[i][df[i]['attention_passed'] == True]
    columns_to_drop = ['assignmentId', 'projectId', 'attention_passed', 'total_time', 'identity_choices', 'ideologies', 'submit_time']
    df[i].drop(columns_to_drop, inplace=True, axis=1)
    print(f"There are {len(df[i])} valid data collected\n")
    df[i].set_index('participantId', inplace=True)
df[0].head()

### Bot detection

In [47]:
def get_bot_detection_num(detection):
    first = detection // 10
    second = detection % 10
    return int(first == 3) + int(second == 3)

ratio_at_least_one_bot_detected = []
ratio_one_bot_detected = []
ratio_two_bots_detected = []

for i in range(3):
    df[i]['bot_detection_num'] = df[i]['bot_detected'].apply(get_bot_detection_num)
    ratio_at_least_one_bot_detected.append(((df[i]['bot_detection_num'] >= 1).sum() + 1) / (len(df[i]) + 1))
    ratio_one_bot_detected.append(((df[i]['bot_detection_num'] == 1).sum() + 1) / (len(df[i]) + 1))
    ratio_two_bots_detected.append(((df[i]['bot_detection_num'] == 2).sum() + 1) / (len(df[i]) + 1))
    print(f"For condition {i}, there are {ratio_at_least_one_bot_detected[i]:.2%} participants who detected at least one bot")
    
# print("\nReasons:")
#     for reason in df['reason'].tolist():
#         if reason and reason != 'None':
#             print(reason)
#     df.drop(['reason', 'bot_detection_num'], axis=1, inplace=True)

For condition 0, there are 38.14% participants who detected at least one bot
For condition 1, there are 36.36% participants who detected at least one bot
For condition 2, there are 100.00% participants who detected at least one bot


## Get reported Ideology Distance

In [16]:
def get_ideology_dis(row):
    additional_answers = row['additional_answers'][0]
    return [abs(additional_answers[1] - additional_answers[0]), abs(additional_answers[1] - abs(additional_answers[2]))]

for i in range(3):
    df[i]['ideology_distance'] = df[i].apply(get_ideology_dis, axis=1)

df[0].head()

IndexError: string index out of range

## Compare distance between two cases
1. When human answers first, calculate the distance between his answer and the other two bots' following answers.
2. If a bot answers first, calculate the distance between his answer and human's following answer

Result should be that given each question, the distance in case 2 is smaller than that in case 1.

Calculate 3 thins for each question:
1. distance in case 1
2. distance in case 2
3. average answer of human if human answers first to examine the result of pilot_2

In [49]:
df[0].head()

,bot_detected,non_ideology_answers,additional_answers,reason,bot_detection_num
participantId,,,,,
077EBB20179F4DDF9DA032AF2B7EE4F4,30,"[{'answers': [-100, 2.1, -100], 'time_to_answe...","[[1, -1.7, -1]]",I just thought it was too inconvenie of a time...,1
08118FE4A7504A34B93DEC7B88ED4EB6,66,"[{'answers': [-100, 1.5500000000000007, -100],...","[[-0.2, 0.7, -0.4]]",,0
0AA37D5D01BA4B76A3FF64C0B1F49B70,66,"[{'answers': [-100, 1.1, -100], 'time_to_answe...","[[0.6, 0.2, -0.2]]",None,0
0B8D09A73ED140CF84C5320DAE27F350,50,"[{'answers': [-100, -3, -100], 'time_to_answer...","[[1, 2, 1]]",None,0
0B97FA950789450AA070261124D188E7,33,"[{'answers': [-100, 1.666666666666667, -100], ...","[[0.1, -0.1, 0.3]]",I just assume that they are bots based on how ...,2


In [43]:
each_answer = [None, None, None]
for i in range(3):
    each_answer[i] = df[i].explode('non_ideology_answers')
    each_answer[i]['answers'] = each_answer[i].apply(lambda row: row['non_ideology_answers']['answers'], axis=1)
    each_answer[i]['idx_of_question'] = each_answer[i].apply(lambda row: row['non_ideology_answers']['idx_of_question'], axis=1)
    each_answer[i]['who_answers_first'] = each_answer[i].apply(lambda row: row['non_ideology_answers']['who_answers_first'], axis=1)
    each_answer[i].drop(['non_ideology_answers', 'additional_answers'], axis=1, inplace=True)
each_answer[0].head()   

TypeError: string indices must be integers

In [8]:
def detect_first_answering_bot(row):
    if row['who_answers_first'] == 0:
        return (row['bot_detected'] // 10) == 3
    else:
        return (row['bot_detected'] % 10) == 3

each_answer['detect_first_answering_bot'] = each_answer.apply(detect_first_answering_bot, axis=1)
each_answer.drop(['bot_detected'], inplace=True, axis=1)
each_answer.head()

,answers,idx_of_question,who_answers_first,detect_first_answering_bot
participantId,,,,
077EBB20179F4DDF9DA032AF2B7EE4F4,"[-100, 2.1, -100]",1,1,False
077EBB20179F4DDF9DA032AF2B7EE4F4,"[-0.6, 1.1, -100]",5,0,True
077EBB20179F4DDF9DA032AF2B7EE4F4,"[-100, 2.3, -100]",0,1,False
077EBB20179F4DDF9DA032AF2B7EE4F4,"[1, 1, -100]",2,0,True
077EBB20179F4DDF9DA032AF2B7EE4F4,"[-100, 2.0500000000000007, 2.1500000000000004]",3,2,False


In [17]:
import random

simulation_times = 100
human_first_answer_distances = []

def calculate_answer_distance(row):
    if row['who_answers_first'] == 0 or row['who_answers_first'] == 2:
        return abs(row['answers'][1] - row['answers'][row['who_answers_first']])
    else:
        return abs(row['answers'][1] - (random.random() * 6 - 3))

for _ in range(simulation_times):
    each_answer['answer_distance'] = each_answer.apply(calculate_answer_distance, axis=1)
    human_first_answer_distances.append(each_answer['answer_distance'][each_answer['who_answers_first'] == 1].mean())

assert len(human_first_answer_distances) == simulation_times, f"Length doesn't match: first {len(human_first_answer_distances)}, second {simulation_times}"

In [19]:
human_first_answer_distance = sum(human_first_answer_distances) / simulation_times
bot_first_answer_distance = each_answer['answer_distance'][each_answer['who_answers_first'] != 1].mean()
bot_first_not_detected_answer_distance = each_answer['answer_distance'][(each_answer['who_answers_first'] != 1) & (each_answer['detect_first_answering_bot'])].mean()
bot_first_detected_answer_distance = each_answer['answer_distance'][(each_answer['who_answers_first'] != 1) & (~each_answer['detect_first_answering_bot'])].mean()
print(f"If human answers first, the average answer distance is {human_first_answer_distance:.2f}")
print(f"If bot answers first, the average answer distance is {bot_first_answer_distance:.2f}")
print(f"If bot is detected, it's {bot_first_detected_answer_distance:.2f}, else it's {bot_first_not_detected_answer_distance:.2f}")

If human answers first, the average answer distance is 2.05
If bot answers first, the average answer distance is 1.76
If bot is detected, it's 1.70, else it's 1.90


In [22]:
bot_first_each_answer = each_answer[each_answer['who_answers_first'] != 1]
each_answer_grouped = bot_first_each_answer.groupby("idx_of_question")
each_answer_grouped['answer_distance'].mean()

idx_of_question
0    1.825000
1    1.952830
2    1.723148
3    1.800000
4    1.415942
5    1.900000
Name: answer_distance, dtype: float64

In [53]:
print(f"Spearman correlation between the first bot's answer and human's answer: {bot_first_each_answer['first_answer'].corr(bot_first_each_answer['human_answer'], method='spearman'):.3}")
print(f"Pearson correlation between the first bot's answer and human's answer: {bot_first_each_answer['first_answer'].corr(bot_first_each_answer['human_answer']):.3}")

bot_first_each_answer_grouped = bot_first_each_answer.groupby('idx_of_question')

def calculate_spearman_correlation(group):
    return group['first_answer'].corr(group['human_answer'], method='spearman')

def calculate_pearson_correlation(group):
    return group['first_answer'].corr(group['human_answer'])

each_idx_spearman_correlation = bot_first_each_answer_grouped.apply(calculate_spearman_correlation)
each_idx_pearson_correlation =   bot_first_each_answer_grouped.apply(calculate_pearson_correlation)
print(each_idx_spearman_correlation, each_idx_pearson_correlation, sep='\n\n')

Spearman correlation between the first bot's answer and human's answer: 0.204
Pearson correlation between the first bot's answer and human's answer: 0.198
idx_of_question
0    0.197300
1    0.020844
2    0.141040
3    0.182601
4    0.479047
5    0.135180
dtype: float64

idx_of_question
0    0.218859
1    0.041136
2    0.113612
3    0.204278
4    0.513295
5    0.095843
dtype: float64


In [54]:
bot_first_each_answer_detection_grouped = bot_first_each_answer.groupby('detect_first_answering_bot')

each_detection_spearman_correlation = bot_first_each_answer_detection_grouped.apply(calculate_spearman_correlation)
each_detection_pearson_correlation = bot_first_each_answer_detection_grouped.apply(calculate_pearson_correlation)
print(each_detection_spearman_correlation, each_detection_pearson_correlation, sep='\n\n')

detect_first_answering_bot
False    0.242244
True     0.101894
dtype: float64

detect_first_answering_bot
False    0.236285
True     0.098929
dtype: float64


In [56]:
bot_first_each_answer_detected = bot_first_each_answer[bot_first_each_answer['detect_first_answering_bot'] == True]

bot_first_each_answer_detected_grouped = bot_first_each_answer_detected.groupby('idx_of_question')

def calculate_spearman_correlation(group):
    return group['first_answer'].corr(group['human_answer'], method='spearman')

def calculate_pearson_correlation(group):
    return group['first_answer'].corr(group['human_answer'])

each_idx_detected_spearman_correlation = bot_first_each_answer_detected_grouped.apply(calculate_spearman_correlation)
each_idx_detected_pearson_correlation = bot_first_each_answer_detected_grouped.apply(calculate_pearson_correlation)
print(each_idx_detected_spearman_correlation, each_idx_detected_pearson_correlation, sep='\n\n')

idx_of_question
0    0.345661
1   -0.188798
2    0.190080
3    0.079669
4    0.317127
5   -0.008411
dtype: float64

idx_of_question
0    0.328745
1   -0.249808
2    0.207524
3    0.099131
4    0.441648
5   -0.071080
dtype: float64


In [27]:
a = "  asdf asd asdf "
b = a.split()
c = b.reverse()
c = list(reversed(b))
c

['asdf', 'asd', 'asdf']